In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

hp = pd.read_csv("./data/HousePrice2.csv")
hp.columns


In [ ]:
htmlColors = pd.read_csv("./data/html_colors.csv", 
                         skiprows = 7,
                        names = ['name1','name2','hex', 'code1','code2','code3'])
htmlColors.head()


In [ ]:
len(hp['Neighborhood'].unique())

In [ ]:
with open ('./data/GmapAPI.txt') as f: 
    api_key = f.readline()
    f.close

In [ ]:
api_key

In [ ]:
import gmaps
gmaps.configure(api_key=api_key)

In [ ]:
locator = Nominatim(user_agent="hi")
ISU = locator.geocode('Ames, IA 50011')
ISU_coord = (ISU.latitude, ISU.longitude)
ISU_coord

In [ ]:

address = hp[['latitude', 'longitude','Neighborhood']]

neighbors = hp.Neighborhood.unique()
np.random.seed(0)
colors = np.random.choice(htmlColors['hex'], size = len(hp['Neighborhood'].unique()), \
                          replace = False)
nbDict = dict(zip(neighbors,colors))

address['Neighborhood_color'] = hp['Neighborhood'].map(nbDict)

address = address.dropna()
address.head()

In [ ]:
fig = gmaps.figure(center = ISU_coord, map_type = 'TERRAIN', zoom_level = 10)
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    address[['latitude','longitude']], 
    hover_text = list(address['Neighborhood']),
    fill_color = list(address['Neighborhood_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig


In [ ]:
fig = gmaps.figure(center = ISU_coord, zoom_level = 10)
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    address[['latitude','longitude']], 
    hover_text = list(address['Neighborhood']),
    fill_color = list(address['Neighborhood_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig



In [ ]:
nbDict2 = dict(zip(colors,neighbors))
htmlColors['Neighborhood'] = htmlColors['hex'].map(nbDict2)
htmlColors.dropna()
